In [1]:
import tensorflow as tf
import cv2
import matplotlib.pyplot as plt
import numpy as np

In [32]:
EDGES = {
    (0, 1): 'm',
    (0, 2): 'c',
    (1, 3): 'm',
    (2, 4): 'c',
    (0, 5): 'm',
    (0, 6): 'c',
    (5, 7): 'm',
    (7, 9): 'm',
    (6, 8): 'c',
    (8, 10): 'c',
    (5, 6): 'y',
    (5, 11): 'm',
    (6, 12): 'c',
    (11, 12): 'y',
    (11, 13): 'm',
    (13, 15): 'm',
    (12, 14): 'c',
    (14, 16): 'c'
}

In [3]:
interpreter = tf.lite.Interpreter(model_path='lite-model_movenet_singlepose_lightning_3.tflite')
interpreter.allocate_tensors()

In [35]:
cap = cv2.VideoCapture(0)
while cap.isOpened():
    ret, frame = cap.read()
    img = frame.copy()
    img = tf.image.resize_with_pad(np.expand_dims(img, axis=0), 192,192)
    input_image = tf.cast(img, dtype=tf.float32)
    input_details = interpreter.get_input_details()
    output_details = interpreter.get_output_details()
    interpreter.set_tensor(input_details[0]['index'], np.array(input_image))
    interpreter.invoke()
    keypoints_with_scores = interpreter.get_tensor(output_details[0]['index'])
    draw_keypoints(frame, keypoints_with_scores, 0.4)
    draw_connections(frame,keypoints_with_scores,EDGES,0.4)
    #print(keypoints_with_scores)
    cv2.imshow('MoveNet Lightning', frame)
    if cv2.waitKey(10) & 0xFF==ord('q'):
        break
        
cap.release()
cv2.destroyAllWindows()

In [14]:
img=frame.copy()
np.expand_dims(img,axis=0).shape

(1, 480, 640, 3)

In [21]:
right_eye=keypoints_with_scores[0][0][2]
right_eye

array([0.26443577, 0.4587251 , 0.6275842 ], dtype=float32)

In [24]:
right_eye_coord=np.array(right_eye[:2]*[480,640]).astype(int)
right_eye_coord

array([126, 293])

In [26]:
shaped = np.squeeze(np.multiply(interpreter.get_tensor(interpreter.get_output_details()[0]['index']), [480,640,1]))

In [27]:
shaped

array([[1.50827522e+02, 3.52420044e+02, 6.31478310e-01],
       [1.23691006e+02, 3.92622452e+02, 8.89943659e-01],
       [1.26929169e+02, 2.93584061e+02, 6.27584219e-01],
       [1.68599453e+02, 4.32411156e+02, 4.97332573e-01],
       [1.74786000e+02, 2.30326519e+02, 6.85736299e-01],
       [3.13227882e+02, 5.76364861e+02, 6.05513573e-01],
       [3.29506416e+02, 9.42217731e+01, 6.80899501e-01],
       [3.28126001e+02, 6.26078568e+02, 6.51599541e-02],
       [3.99054079e+02, 8.78692627e+00, 4.71896119e-02],
       [2.66109295e+02, 5.16138535e+02, 3.76869738e-03],
       [2.07124658e+02, 2.00896835e+02, 2.27936562e-02],
       [1.41620622e+02, 4.86093864e+02, 1.17531894e-02],
       [4.83138542e+02, 1.41929698e+01, 1.06220450e-02],
       [3.20947495e+02, 6.20546875e+02, 2.32640877e-02],
       [1.67448220e+02, 2.16257324e+02, 1.94503292e-02],
       [1.87393727e+02, 4.74290390e+02, 1.98856313e-02],
       [1.85094938e+02, 4.24135742e+02, 1.05623584e-02]])

In [31]:
def draw_keypoints(frame, keypoints, confidence_threshold):
    y, x, c = frame.shape
    shaped = np.squeeze(np.multiply(keypoints, [y,x,1]))
    
    for kp in shaped:
        ky, kx, kp_conf = kp
        if kp_conf > confidence_threshold:
            cv2.circle(frame, (int(kx), int(ky)), 5, (0,255,0), -1) 

In [34]:
def draw_connections(frame, keypoints, edges, confidence_threshold):
    y, x, c = frame.shape
    shaped = np.squeeze(np.multiply(keypoints, [y,x,1]))
    
    for edge, color in edges.items():
        p1, p2 = edge
        y1, x1, c1 = shaped[p1]
        y2, x2, c2 = shaped[p2]
        
        if (c1 > confidence_threshold) & (c2 > confidence_threshold):      
            cv2.line(frame, (int(x1), int(y1)), (int(x2), int(y2)), (0,0,255), 2)